In [76]:
import pandas as pd
import plotly.express as px

In [77]:
df = pd.read_csv('../data/Fluoridation by postcode - Sheet1.csv')

In [78]:
df.dtypes

cartodb_id          int64
melbourne            bool
postcode            int64
id                  int64
fluoride_level     object
water_company      object
townsuburb         object
lat               float64
lon               float64
dtype: object

In [79]:
df['fluoride_level'].unique() 

array(['Fluoridated', 'Not fluoridated', 'Partly fluoridated',
       'Natural fluoride'], dtype=object)

In [80]:
df['melbourne'].unique()

array([False,  True])

In [81]:
df[df.melbourne == True]

,cartodb_id,melbourne,postcode,id,fluoride_level,water_company,townsuburb,lat,lon
1,1,True,3067,1,Fluoridated,City West Water,ABBOTSFORD,-37.803000,145.002000
2,2,True,3040,2,Fluoridated,City West Water,ABERFELDIE,-37.762000,144.901000
7,7,True,3042,7,Fluoridated,City West Water,AIRPORT WEST,-37.724000,144.879000
8,8,True,3021,8,Fluoridated,City West Water,ALBANVALE,-37.746000,144.765000
9,9,True,3206,9,Fluoridated,South East Water,ALBERT PARK,-37.842000,144.950000
...,...,...,...,...,...,...,...,...,...
1048,1049,True,3013,1049,Fluoridated,City West Water,YARRAVILLE,-37.817000,144.890000
1049,1050,True,3013,1050,Fluoridated,City West Water,YARRAVILLE WEST,-37.815066,144.885260
1053,1054,True,3139,1054,Partly fluoridated,Yarra Valley Water,YELLINGBO,-37.810000,145.510000
1055,1056,True,3770,1056,Fluoridated,Yarra Valley Water,YERING,-37.700221,145.358251


In [82]:
fig = px.bar(
    df.groupby(['fluoride_level', 'melbourne']).size().reset_index(name='count'),
    x='fluoride_level',
    y='count',
    color='melbourne',
)


fig.show()

In [83]:
df.groupby(['melbourne', 'townsuburb']).agg({
    'water_company': 'nunique',
    'fluoride_level': lambda x: x[x=='Fluoridated'].count(),
}).sort_values('water_company', ascending=False)

water_company  fluoride_level
melbourne townsuburb                                   
True      UPWEY                       2               2
          COBURG                      2               2
          TULLAMARINE                 2               2
          OAKLEIGH                    2               2
          ARMADALE                    2               2
...                                 ...             ...
False     MERRICKS                    1               1
          MERRICKS BEACH              1               1
          MERRIGUM                    1               1
          MERRIJIG                    1               0
True      YUROKE                      1               1

[1046 rows x 2 columns]

In [84]:
df.iloc[588:590,:]

,cartodb_id,melbourne,postcode,id,fluoride_level,water_company,townsuburb,lat,lon
588,587,True,3000,587,Fluoridated,City West Water,MELBOURNE,-37.813628,144.963058
589,588,True,3004,588,Fluoridated,City West Water,MELBOURNE,-37.838525,144.975608


In [85]:
df.query('postcode == 3000')['lat']

588   -37.813628
Name: lat, dtype: float64

In [86]:
type(df.query('postcode == 3000')['lat'].astype(float))

pandas.core.series.Series

In [87]:
df.shape

(1059, 9)

In [88]:
# cptlized_surburbs = df.apply(lambda x: 
#                 x['townsuburb'].capitalize() + ' ' + '(' + str(x['postcode']) + ')'
#                 if x['townsuburb'] == 'MELBOURNE'
#                 else x['townsuburb'].capitalize(),
#                 axis=1)
filtered_str = filter(lambda x: 
                                'Melbourne' in x,
                        df.apply(lambda x: 
                                        x['townsuburb'].capitalize() + ' ' + '(' + str(x['postcode']) + ')'
                                        if x['townsuburb'] == 'MELBOURNE'
                                        else x['townsuburb'].capitalize(),
                                        axis=1))



list(filtered_str)

['Melbourne (3000)', 'Melbourne (3004)', 'Melbourne airport']

In [113]:
df['water_company'].unique()

array(['Barwon Water', 'City West Water', 'Central Highlands Water',
       'South Gippsland Water', 'South East Water',
       'Goulburn Valley Water', 'Wannon Water', 'Yarra Valley Water',
       'Westernport Water', 'GWMWater', 'Coliban Water', 'Western Water',
       'East Gippsland Water', 'North East Water', 'Lower Murray Water',
       'Gippsland Water', 'Mount Buller and Mount Stirling ARMB',
       'Mount Hotham ARMB'], dtype=object)

In [114]:
fig = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon',
    color='water_company',
    mapbox_style='open-street-map',
    zoom=8,
    center={'lat': df.query('postcode == 3000')['lat'].values[0], 
            'lon': df.query('postcode == 3000')['lon'].values[0]},
    opacity=0.8,
    hover_name='townsuburb',
    hover_data=['water_company'],
    color_discrete_map={'Fluoridated': 'red', 'Not fluoridated': 'blue', 'Partly fluoridated': 'black'},
)
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    width=1280,
    height=720,
    )

fig.show()

In [ ]:
geo_df.index

Index(['11-Sault-au-Récollet', '12-Saint-Sulpice', '13-Ahuntsic',
       '14-Bordeaux-Cartierville', '21-Ouest', '22-Est', '23-Centre',
       '31-Darlington', '32-Côte-des-Neiges', '33-Snowdon',
       '34-Notre-Dame-de-Grâce', '35-Loyola', '41-du Canal',
       '42-J.-Émery-Provost', '43-Fort-Rolland', '51-Sault-Saint-Louis',
       '52-Cecil-P.-Newman', '61-Pierre-Foretier', '62-Denis-Benjamin-Viger',
       '63-Jacques-Bizard', '64-Sainte-Geneviève', '71-Tétreaultville',
       '72-MaisonneuveLongue-Pointe', '73-Hochelaga', '74-Louis-Riel',
       '81-Marie-Clarac', '82-Ovide-Clermont', '91-Claude-Ryan',
       '92-Joseph-Beaubien', '93-Robert-Bourassa', '94-Jeanne-Sauvé',
       '101-Bois-de-Liesse', '102-Cap-Saint-Jacques', '111-Mile-End',
       '113-Jeanne-Mance', '121-La Pointe-aux-Prairies',
       '122-Pointe-aux-Trembles', '123-Rivière-des-Prairies',
       '131-Saint-Édouard', '132-Étienne-Desmarteau', '133-Vieux-Rosemont',
       '134-Marie-Victorin', '141-Côte-de-Liesse'

In [ ]:
gpd.read_file('https://raw.githubusercontent.com/codeforgermany/click_that_hood/main/public/data/melbourne.geojson')

,name,cartodb_id,created_at,updated_at,geometry
0,Cremorne,31,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((144.98905 -37.82341, 144.98793..."
1,Caulfield,56,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((145.01631 -37.87916, 145.01495..."
2,Carnegie,58,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((145.04613 -37.88727, 145.04495..."
3,Kingsville,107,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((144.88543 -37.80259, 144.87400..."
4,South Kingsville,111,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((144.86553 -37.83306, 144.86566..."
...,...,...,...,...,...
117,Altona North,117,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((144.85293 -37.85240, 144.85333..."
118,Altona,118,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((144.85293 -37.85240, 144.84505..."
119,Altona Meadows,119,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((144.80832 -37.88196, 144.80521..."
120,Seabrook,121,2015-03-10 04:37:29+00:00,2015-03-10 04:37:29+00:00,"MULTIPOLYGON (((144.76670 -37.88393, 144.76577..."
